# MNIST dataset

In [14]:
import numpy as np
import tensorflow as tf
import tensorflow.keras as keras
import tensorflow.keras.layers as tf_layers
from tensorflow.keras.datasets.mnist import load_data as load_data_MNIST
from einops import rearrange

import dgates

## Loading data

In [2]:
(x_train, y_train), (x_test, y_test) = load_data_MNIST()

x_train = rearrange(x_train, 'i h w -> i (h w)')
x_test = rearrange(x_test, 'i h w -> i (h w)')

features = x_test > 100
outputs = (y_test == 5).reshape(-1, 1)

## Diff Gates

In [6]:
network = dgates.Network(784, [40, 50, 50, 40, 1], connections_rate=5)

In [7]:
loss, accuracy = network.evaluate_real(features, outputs)
print(f'Loss: {loss:.3f} Accuracy: {accuracy:.3f}')

Predicting: 100%|██████████| 10000/10000 [01:37<00:00, 102.14it/s]

Loss: 0.251 Accuracy: 0.089


In [8]:
loss, accuracy = network.evaluate_boolean(features, outputs)
print(f'Loss: {loss:.3f} Accuracy: {accuracy:.3f}')

Predicting: 100%|██████████| 10000/10000 [00:03<00:00, 3017.22it/s]

Loss: 0.903 Accuracy: 0.097


In [12]:
network.fit(features, outputs, epochs=1, learning_rate=0.1)

Epoch   1: 100%|██████████| 10000/10000 [10:48<00:00, 15.43it/s]


In [14]:
loss, accuracy = network.evaluate_real(features, outputs)
print(f'Loss: {loss:.3f} Accuracy: {accuracy:.3f}')

Predicting: 100%|██████████| 10000/10000 [01:39<00:00, 100.74it/s]

Loss: 0.081 Accuracy: 0.911


In [13]:
loss, accuracy = network.evaluate_boolean(features, outputs)
print(f'Loss: {loss:.3f} Accuracy: {accuracy:.3f}')

Predicting: 100%|██████████| 10000/10000 [00:03<00:00, 2969.71it/s]

Loss: 0.089 Accuracy: 0.911


## Gates

In [3]:
from gates import gates
from gates import optimizers as gates_optimizers
from gates.optimizers import genetic as gates_genetic

In [9]:
network = gates.Network(784, [40, 50, 50, 40, 1])

In [10]:
optimizer = gates_optimizers.GeneticOptimizer(
    fitness=gates_genetic.LowestMseFitness(),
    crossing=gates_genetic.MultiPointCrossing(4),
    mutation=gates_genetic.NRandomChangesMutation(6),
    selection=gates_genetic.RouletteWheelSelection(),
    cross_prob=0.7,
    mutate_prob=0.2,
    n_chromosomes=20,
    n_generations=100,
    n_elite=2,
    patience=10,
)

optimizer.optimize(network, features, outputs)

Optimizing 905 parameters


Evolution:  10%|█         | 10/100 [02:41<24:09, 16.10s/it, fitness: 11.210762331838565, mse: 0.089, accuracy: 0.911]


In [11]:
loss, accuracy = network.evaluate(features, outputs)
print(f'Loss: {loss:.3f} Accuracy: {accuracy:.3f}')

Loss: 0.089 Accuracy: 0.911


## Neural Network

In [23]:
network = keras.models.Sequential([
    tf_layers.InputLayer((784,)),
    tf_layers.Dense(40, activation='relu'),
    tf_layers.Dense(50, activation='relu'),
    tf_layers.Dense(50, activation='relu'),
    tf_layers.Dense(40, activation='relu'),
    tf_layers.Dense(1 , activation=None)
])

In [24]:
network.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_10 (Dense)            (None, 40)                31400     
                                                                 
 dense_11 (Dense)            (None, 50)                2050      
                                                                 
 dense_12 (Dense)            (None, 50)                2550      
                                                                 
 dense_13 (Dense)            (None, 40)                2040      
                                                                 
 dense_14 (Dense)            (None, 1)                 41        
                                                                 
Total params: 38,081
Trainable params: 38,081
Non-trainable params: 0
_________________________________________________________________


In [25]:
network.compile(optimizer='adam', loss='mse', metrics=['binary_accuracy'])

In [26]:
network.fit(
    features,
    outputs,
    validation_data=(features, outputs),
    epochs=10,
    batch_size=1,
)

Epoch 1/10
10000/10000 [==============================] - 10s 1ms/step - loss: 0.0279 - binary_accuracy: 0.9672 - val_loss: 0.0140 - val_binary_accuracy: 0.9854
Epoch 2/10
10000/10000 [==============================] - 10s 1ms/step - loss: 0.0143 - binary_accuracy: 0.9853 - val_loss: 0.0146 - val_binary_accuracy: 0.9857
Epoch 3/10
10000/10000 [==============================] - 10s 1ms/step - loss: 0.0107 - binary_accuracy: 0.9875 - val_loss: 0.0052 - val_binary_accuracy: 0.9947
Epoch 4/10
10000/10000 [==============================] - 10s 1ms/step - loss: 0.0073 - binary_accuracy: 0.9913 - val_loss: 0.0044 - val_binary_accuracy: 0.9950
Epoch 5/10
10000/10000 [==============================] - 10s 1ms/step - loss: 0.0058 - binary_accuracy: 0.9937 - val_loss: 0.0051 - val_binary_accuracy: 0.9939
Epoch 6/10
10000/10000 [==============================] - 10s 1ms/step - loss: 0.0062 - binary_accuracy: 0.9931 - val_loss: 0.0046 - val_binary_accuracy: 0.9949
Epoch 7/10
10000/10000 [==========

In [27]:
loss, accuracy = network.evaluate(features, outputs, verbose=0)
print(f'Loss: {loss:.3f} Accuracy: {accuracy:.3f}')

Loss: 0.002 Accuracy: 0.998
